# 결측치 (missing value) 처리

In [5]:
import numpy as np
import pandas as pd

In [6]:
df = pd.DataFrame([
    [42, 'male', 12, 'reading', 'class2'],
    [35, 'unknown', 3, 'cooking', 'class1'],
    [1000, 'female', 7, 'cycling', 'class3'],
    [1000, 'unknown', 21, 'unknown', 'unknown']
])
df.columns = ['age',
             'gender',
             'month_birth',
             'hobby',
             'target']

In [7]:
df

,age,gender,month_birth,hobby,target
0,42,male,12,reading,class2
1,35,unknown,3,cooking,class1
2,1000,female,7,cycling,class3
3,1000,unknown,21,unknown,unknown


In [8]:
# 각 열의 유니크 값 확인
df['age'].unique()
# df['gender'].unique()
# df['month_birth'].unique()
# df['hobby'].unique()
# df['target'].unique()

array([  42,   35, 1000], dtype=int64)

In [9]:
# 결측치 처리
df.loc[df['age']>150, ['age']] = np.nan
df.loc[df['gender']=='unknown', ['gender']] = np.nan
df.loc[df['month_birth']>12, ['month_birth']] = np.nan
df.loc[df['hobby']=='unknown', ['hobby']] = np.nan
df.loc[df['target']=='unknown', ['target']] = np.nan
df

,age,gender,month_birth,hobby,target
0,42.0,male,12.0,reading,class2
1,35.0,NaN,3.0,cooking,class1
2,NaN,female,7.0,cycling,class3
3,NaN,NaN,NaN,NaN,NaN


In [10]:
# 각 열의 결측치 개수 확인
df.isnull().sum()

age            2
gender         2
month_birth    1
hobby          1
target         1
dtype: int64

In [11]:
# 결측치 포함한 행 삭제
df2 = df.dropna(axis=0)
df2

,age,gender,month_birth,hobby,target
0,42.0,male,12.0,reading,class2


In [12]:
# 결측치 포함한 열 삭제
df3 = df.dropna(axis=1)
df3

""
0
1
2
3


In [13]:
# 모든 값이 결측치인 행 삭제
df4 = df.dropna(how='all')
df4

,age,gender,month_birth,hobby,target
0,42.0,male,12.0,reading,class2
1,35.0,NaN,3.0,cooking,class1
2,NaN,female,7.0,cycling,class3


In [14]:
# 값이 2개 미만인 행 삭제
df5 = df.dropna(thresh=2)
df5

,age,gender,month_birth,hobby,target
0,42.0,male,12.0,reading,class2
1,35.0,NaN,3.0,cooking,class1
2,NaN,female,7.0,cycling,class3


In [15]:
# 특정 열에 결측치가 있는 경우 행 삭제
df6 = df.dropna(subset=['gender'])
df6

,age,gender,month_birth,hobby,target
0,42.0,male,12.0,reading,class2
2,NaN,female,7.0,cycling,class3


In [16]:
# 결측치 대체하기
alter_values = {'age': 0,
                'gender': 'U',
                'month_birth' : 0,
                'hobby' : 'U',
                'target' : 'class4'}
df7 = df.fillna(value = alter_values)
df7

,age,gender,month_birth,hobby,target
0,42.0,male,12.0,reading,class2
1,35.0,U,3.0,cooking,class1
2,0.0,female,7.0,cycling,class3
3,0.0,U,0.0,U,class4


# 클래스 라벨 설정

In [17]:
from sklearn.preprocessing import LabelEncoder

In [18]:
df8 = df7
class_label = LabelEncoder()
data_value = df8['target'].values
y_new = class_label.fit_transform(data_value)
y_new

array([1, 0, 2, 3])

In [21]:
df8['target'] = y_new
df8

,age,gender,month_birth,hobby,target
0,42.0,male,12.0,reading,1
1,35.0,U,3.0,cooking,0
2,0.0,female,7.0,cycling,2
3,0.0,U,0.0,U,3


## 다시 원래대로 돌리기

In [23]:
y_ori = class_label.inverse_transform(y_new)
df8['target'] = y_ori
df8

,age,gender,month_birth,hobby,target
0,42.0,male,12.0,reading,class2
1,35.0,U,3.0,cooking,class1
2,0.0,female,7.0,cycling,class3
3,0.0,U,0.0,U,class4


## 사이킷 런 사용하지 않고 클래스 라벨링

In [24]:
y_arr = df8['target'].values
y_arr.sort()

num_y = 0
dic_y = {}
for ith_y in y_arr:
    dic_y[ith_y] = num_y
    num_y += 1
dic_y

{'class1': 0, 'class2': 1, 'class3': 2, 'class4': 3}

In [33]:
df8['target'] = df8['target'].replace(dic_y)
df8

,age,gender,month_birth,hobby,target
0,42.0,male,12.0,reading,0
1,35.0,U,3.0,cooking,1
2,0.0,female,7.0,cycling,2
3,0.0,U,0.0,U,3


# 원-핫 인코딩

- 클래스 라벨링의 또다른 방법
- 0, 1만을 사용한 벡터를 이용해 데이터값 나타냄
- 더미 변수 (dummy variable) 라고도 함

In [28]:
df9 = df8
df9['target'] = df9['target'].astype(str) # 타깃 데이터 값을 문자열로 바꿈
df10 = pd.get_dummies(df9['target'])
print(df10)
# 타깃0 은 (1, 0, 0, 0), 타깃1 은 (0, 1, 0, 0) 처럼 표현됨

   0  1  2  3
0  1  0  0  0
1  0  1  0  0
2  0  0  1  0
3  0  0  0  1


In [29]:
# 길이가 3 인 벡터로 표현 (타깃0이 (0, 0, 0) 으로 표현)
df9['target'] = df9['target'].astype(str)
df11 = pd.get_dummies(df9['target'], drop_first = True)
print(df11)

   1  2  3
0  0  0  0
1  1  0  0
2  0  1  0
3  0  0  1


In [32]:
# 모든 열 원-핫 인코딩
df12 = df8
df13 = pd.get_dummies(df12)
df13

,age,month_birth,gender_U,gender_female,gender_male,hobby_U,hobby_cooking,hobby_cycling,hobby_reading,target_0,target_1,target_2,target_3
0,42.0,12.0,0,0,1,0,0,0,1,1,0,0,0
1,35.0,3.0,1,0,0,0,1,0,0,0,1,0,0
2,0.0,7.0,0,1,0,0,0,1,0,0,0,1,0
3,0.0,0.0,1,0,0,1,0,0,0,0,0,0,1


(1) 사이킷런 라이브러리를 이용한 원-핫 인코딩

In [34]:
# '태어난 달' 열의 데이터값 원-핫 인코딩
from sklearn.preprocessing import OneHotEncoder

hot_encoder = OneHotEncoder()
y = df7[['target']]
y_hot = hot_encoder.fit_transform(y)
print(y_hot.toarray())

[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]


(2) 텐서플로 라이브러리를 이용한 원-핫 인코딩

In [35]:
from tensorflow.keras.utils import to_categorical

y_hotec = to_categorical(y)
print(y_hotec)

[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]


# 데이터 스케일링

(1) 표준화 스케일링
- 평균 0, 표준 편차 1이 되도록 변경
- (x_i - x_mean)/x_sigma

In [38]:
# '태어난 달'의 열 표준화
from sklearn.preprocessing import StandardScaler

std = StandardScaler()
std.fit(df8[['month_birth']]) # 표준화 스케일러에 month_birth 적합시킴
x_std = std.transform(df8[['month_birth']])    # 적합된 표준화 스케일러를 기준으로 month_birth 열 데이터 값 변형

In [39]:
# 위에서 아래 두 줄 코드 간략화
x_std2 = std.fit_transform(df8[['month_birth']])
x_std2

array([[ 1.44444444],
       [-0.55555556],
       [ 0.33333333],
       [-1.22222222]])

In [44]:
# 표준화 후 평균, 표준편차 확인
np.mean(x_std)

-5.551115123125783e-17

In [42]:
np.std(x_std)

1.0

(2) 로버스트 스케일링
- 표준화 스케일링을 변형한 방법
- 중앙값 (median)과 사분위수 (quantile) 사용
- 극단값의 영향을 거의 받지 않음
- (x_i - q_2)/(q_3 - q_1)
- q_1: 1사분위수, q_2: 중위수, q_3: 3사분위수

In [45]:
# '태어난 달'의 열 표준화
from sklearn.preprocessing import RobustScaler

robust = RobustScaler()
robust.fit(df8[['month_birth']])                    # 스케일링하려는 데이터 프레임의 열을 이용해 적합함
x_robust = robust.transform(df8[['month_birth']])   # 적합된 로버스트 스케일러를 기반으로 데이터값 변환
x_robust

array([[ 1.16666667],
       [-0.33333333],
       [ 0.33333333],
       [-0.83333333]])

(3) 최대-최소 스케일링
- 최대값 1, 최소값 0으로 스케일링
- (x_i - min(x))/(max(x) - min(x))

In [48]:
# '태어난 달'의 열 표준화
from sklearn.preprocessing import MinMaxScaler

minmax = MinMaxScaler()
minmax.fit(df8[['month_birth']])
x_minmax = minmax.transform(df8[['month_birth']])
x_minmax

array([[1.        ],
       [0.25      ],
       [0.58333333],
       [0.        ]])

(4) 노멀 스케일링
- 벡터의 유클리디안 길이가 1이 되도록 변경
- 길이 상관없이 방향(각도)만 고려할 때 사용
- 노멀 스케일은 행(row) 기준
- normalization이라고도 함
- new(x_i) = x_i/sqrt(x_i^2 + y_i^2 + z_i^2)

In [51]:
# '태어난 달'의 표준화
from sklearn.preprocessing import Normalizer

normal = Normalizer()
normal.fit(df8[['age', 'month_birth']])
x_normal = normal.transform(df8[['age', 'month_birth']])
x_normal

array([[0.96152395, 0.27472113],
       [0.99634665, 0.08540114],
       [0.        , 1.        ],
       [0.        , 0.        ]])

In [54]:
'''# 표준화 스케일링
from sklearn.preprocessing import StandardScaler

stand_scale = StandardScaler()
x_train_std = stand_scale.fit_transform(x_train)
x_test_std = stand_scale.transform(x_test)'''

"""데이터 스케일링 과정에서 fit() 메소드는 트레이닝 데이터 셋에 대해서만 사용.
테스트 데이터 셋에는 transform()만 사용"""

'데이터 스케일링 과정에서 fit() 메소드는 트레이닝 데이터 셋에 대해서만 사용.\n테스트 데이터 셋에는 transform()만 사용'